# Transfer Learning

В этом блокноте вы научитесь использовать предобученные сети для решения задач в области компьютерного зрения. В частности, вы будете использовать сети, обученные на [ImageNet](http://www.image-net.org/) [доступно из torchvision](https://pytorch.org/vision/main/models.html). 

ImageNet — это обширный набор данных с более чем 1 миллионом размеченных изображений и 1000 категорий. Он используется для обучения глубоких свёрточных нейронных сетей. Свёрточные сети мы подробнее рассмотрим на следующей лабораторной.

После обучения эти модели работают удивительно хорошо для извлечения признаков из изображений, на которых они не были обучены. Использование заранее обученной сети на изображениях, которых нет в обучающем наборе, называется transfer learning (перенос обучения). Здесь мы будем использовать transfer learning, чтобы обучить сеть, которая может классифицировать фотографии кошек и собак с почти идеальной точностью.

С помощью `torchvision.models` вы можете загрузить эти предобученные сети и использовать их в своих приложениях. Добавим `models` в наши импорты.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Большинство предобученных моделей требуют, чтобы входные данные были изображениями размером 224x224. Также нам нужно использовать ту же нормализацию, что использовалась при обучении моделей. Каждый цветовой канал нормализовался отдельно, средние значения составляют `[0.485, 0.456, 0.406]`, а стандартные отклонения — `[0.229, 0.224, 0.225]`.

In [ ]:
## TODO

data_dir = 'Cat_Dog_data'

# TODO: Определите преобразования для обучающего и тестового наборов данных
train_transforms = 

test_transforms = 

# Примените преобразования, создайте объекты датасетов и загрузчиков данных для обучающего и тестового наборов
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

Загрузим модель, такую как [DenseNet](https://pytorch.org/vision/main/models/generated/torchvision.models.densenet121.html). Давайте выведем архитектуру модели, чтобы увидеть ее составные блоки.

In [ ]:
model = models.densenet121(pretrained=True)
model

Эта модель состоит из двух основных частей: "извлекателя" признаков и классификатора. "Извлекатель" признаков — это стек свёрточных слоёв, который в целом формирует набор признаков, который можно передать классификатору. Часть классификатора — это один полносвязный слой `(classifier): Linear(in_features=1024, out_features=1000)`. Этот слой был обучен на наборе данных ImageNet, поэтому он не подойдёт для нашей конкретной задачи. Это означает, что нам нужно заменить классификатор, но признаки будут работать идеально сами по себе. В общем, заранее обученные сети можно понимать как удивительно хорошие детекторы признаков, которые могут использоваться в качестве входных данных для простых классификаторов.

In [ ]:
# Замораживаем параметры, чтобы мы не могли выполнить обратное распространение ошибки через них
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

Затем нам необходимо обучить классификатор. Однако теперь мы используем **очень глубокую** нейронную сеть. Если вы попытаетесь обучить её на центральном процессоре, как обычно, это займет очень много времени. Вместо этого мы будем использовать графический процессор (GPU) для выполнения вычислений. Вычисления линейной алгебры выполняются параллельно на GPU, что приводит к увеличению скорости обучения в 100 раз. Также возможно обучение на нескольких GPU, что ещё больше сокращает время обучения.

PyTorch, наряду с практически всеми другими фреймворками глубокого обучения, использует [CUDA](https://developer.nvidia.com/cuda-zone) для эффективного выполнения прямых и обратных проходов на GPU. В PyTorch вы перемещаете параметры вашей модели и другие тензоры в память GPU, используя `model.to('cuda')`. Вы можете перемещать их обратно с GPU с помощью `model.to('cpu')`, что вам часто нужно делать, когда вам нужно работать с выходом сети вне PyTorch. В качестве демонстрации увеличенной скорости сравним, сколько времени требуется для выполнения прямого и обратного прохода с помощью и без графического процессора.

In [ ]:
import time

In [ ]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Обучаем только параметры классификатора, параметры извлечения признаков заморожены
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Перемещаем тензоры входных данных и меток на GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Возможно писать код, не зависимый от устройства, который будет автоматически использовать CUDA, если оно включено, следующим образом:
```python
# в начале скрипта
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# а затем каждый раз, когда вы получаете новый Тензор или Модуль, выполните следующий вызов
# это не будет приводить к копированию, если они уже находятся на желаемом устройстве
input = data.to(device)
model = MyModule(...).to(device)
```

Закончите обучение модели самостоятельно. Процесс такой же, как и раньше, за исключением того, что теперь ваша модель намного мощнее. Вы должны легко получить точность выше 95%.

>**Упражнение:** Обучите предобученную модель на классификацию изображений кошек и собак. Продолжайте работать с моделью DenseNet или попробуйте ResNet — это тоже хорошая модель для первого ознакомления. Убедитесь, что вы обучаете только классификатор, а параметры извлечения признаков заморожены.

In [ ]:
## TODO
